## IMPUTING MISSING VALUES

In [10]:
import sqlite3
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing

In [11]:
df = pd.read_pickle('insurance_cleaned.pkl')
df.head()

,ed_degree,month_salary,living_area,has_children,customer_monetary_value,claims_rate,premium_motor,premium_house,premium_health,premium_life,premium_work,years_as_customer
c_id,,,,,,,,,,,,
1,2 - High School,2177.0,1.0,1.0,380.97,0.39,375.85,79.45,146.36,47.01,16.89,31.0
2,2 - High School,677.0,4.0,1.0,-131.13,1.12,77.46,416.20,116.69,194.48,106.13,35.0
3,1 - Basic,2277.0,3.0,0.0,504.67,0.28,206.15,224.50,124.58,86.35,99.02,25.0
4,3 - BSc/MSc,1099.0,4.0,1.0,-16.99,0.99,182.48,43.35,311.17,35.34,28.34,26.0
5,3 - BSc/MSc,1763.0,4.0,1.0,35.23,0.90,338.62,47.80,182.59,18.78,41.45,30.0


### Splitting data
#### In training data to train K-neighbors and test data
    Training data is made up only of observations with missing values and test data with the remaining observations.

In [12]:
train_data = df.dropna()

In [13]:
test_data = df[pd.isnull(df).any(axis=1)]
test_data.head()

,ed_degree,month_salary,living_area,has_children,customer_monetary_value,claims_rate,premium_motor,premium_house,premium_health,premium_life,premium_work,years_as_customer
c_id,,,,,,,,,,,,
22,2 - High School,NaN,3.0,1.0,111.37,0.80,407.52,111.70,100.13,24.67,30.34,39.0
69,2 - High School,1399.0,4.0,1.0,476.65,0.33,330.73,186.70,211.15,NaN,15.78,33.0
139,2 - High School,2538.0,2.0,1.0,149.59,0.69,194.26,305.60,NaN,37.34,17.67,37.0
144,3 - BSc/MSc,NaN,4.0,1.0,-42.45,1.02,146.36,353.95,220.04,121.80,42.01,20.0
172,3 - BSc/MSc,1086.0,4.0,1.0,NaN,NaN,378.07,78.90,166.81,6.89,18.45,26.0


In [14]:
len(test_data)+len(train_data) == len(df)

True

In [15]:
df_labels = train_data[['ed_degree', 'has_children', 'living_area']]

#### Label Encoding

In [16]:
le_degree = preprocessing.LabelEncoder()
le_area = preprocessing.LabelEncoder()
le_children = preprocessing.LabelEncoder()

In [17]:
df_labels['degree_encoded'] = le_degree.fit_transform(df_labels.ed_degree)
df_labels['area_encoded'] = le_area.fit_transform(df_labels.living_area)
df_labels['children_encoded'] = le_children.fit_transform(df_labels.has_children)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

#### Hot Encoding

In [18]:
degree_he = preprocessing.OneHotEncoder()
has_children_he = preprocessing.OneHotEncoder()
living_area_he = preprocessing.OneHotEncoder()

In [19]:
df_labels.head()

,ed_degree,has_children,living_area,degree_encoded,area_encoded,children_encoded
c_id,,,,,,
1,2 - High School,1.0,1.0,1,0,1
2,2 - High School,1.0,4.0,1,3,1
3,1 - Basic,0.0,3.0,0,2,0
4,3 - BSc/MSc,1.0,4.0,2,3,1
5,3 - BSc/MSc,1.0,4.0,2,3,1


In [20]:
degrees = degree_he.fit_transform(df_labels.degree_encoded.values.reshape(-1,1)).toarray()

In [21]:
df_labels['degree_he'] = pd.Categorical(df_labels['ed_degree'])
df_labels['area_he'] = pd.Categorical(df_labels['living_area'])
df_labels['children_he'] = pd.Categorical(df_labels['has_children'])

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [22]:
df_degree = pd.get_dummies(df_labels['degree_he'], prefix = 'categ')
df_area = pd.get_dummies(df_labels['area_he'], prefix = 'categ')
df_child = pd.get_dummies(df_labels['children_he'], prefix = 'categ')

#### Renaming the columns

In [23]:
df_child.columns = ['No Children', 'Children']
df_degree.columns = ['Basic', 'High School', 'BSc/MSc', 'PhD']
df_area.columns = ['A1', 'A2', 'A3', 'A4']
new_df = pd.concat([df_degree, df_child, df_area], axis = 1)

In [24]:
new_df.head()

,Basic,High School,BSc/MSc,PhD,No Children,Children,A1,A2,A3,A4
c_id,,,,,,,,,,
1,0,1,0,0,0,1,1,0,0,0
2,0,1,0,0,0,1,0,0,0,1
3,1,0,0,0,1,0,0,0,1,0
4,0,0,1,0,0,1,0,0,0,1
5,0,0,1,0,0,1,0,0,0,1


In [25]:
df_numeric = train_data[['month_salary', 'customer_monetary_value', 'claims_rate', 'premium_motor', 'premium_house', 'premium_health', 'premium_life', 'premium_work', 'years_as_customer']]
df_encoded = new_df.copy()

#### Scaling Data

In [30]:
scaler = preprocessing.StandardScaler()
scaled_df_numeric = scaler.fit_transform(df_numeric)
scaled_df_numeric = pd.DataFrame(scaled_df_numeric, columns = df_numeric.columns, index = df_numeric.index)

In [32]:
new_df.head()

,Basic,High School,BSc/MSc,PhD,No Children,Children,A1,A2,A3,A4
c_id,,,,,,,,,,
1,0,1,0,0,0,1,1,0,0,0
2,0,1,0,0,0,1,0,0,0,1
3,1,0,0,0,1,0,0,0,1,0
4,0,0,1,0,0,1,0,0,0,1
5,0,0,1,0,0,1,0,0,0,1


In [33]:
df_scaled_labeled = pd.concat([scaled_df_numeric, new_df], axis = 1)

In [34]:
df_scaled_labeled.head()

,month_salary,customer_monetary_value,claims_rate,premium_motor,premium_house,premium_health,premium_life,premium_work,years_as_customer,Basic,High School,BSc/MSc,PhD,No Children,Children,A1,A2,A3,A4
c_id,,,,,,,,,,,,,,,,,,,
1,-0.324942,0.176712,-0.220951,0.431656,-0.514009,-0.221117,0.104866,-0.509725,0.011972,0,1,0,0,0,1,1,0,0,0
2,-1.844276,-0.304750,0.268088,-1.218462,0.806756,-0.505688,3.202084,1.355407,0.019693,0,1,0,0,0,1,0,0,0,1
3,-0.223653,0.293012,-0.294642,-0.506797,0.054890,-0.430013,0.931099,1.206807,0.000392,1,0,0,0,1,0,0,0,1,0
4,-1.416837,-0.197439,0.180999,-0.637694,-0.655597,1.359612,-0.140231,-0.270418,0.002322,0,0,1,0,0,1,0,0,0,1
5,-0.744279,-0.148343,0.120706,0.225771,-0.638144,0.126373,-0.488030,0.003584,0.010042,0,0,1,0,0,1,0,0,0,1


In [35]:
df_kneighbors = pd.DataFrame([df_scaled_labeled.iloc[:,i].apply(lambda x: int(x*1000)) for i,j in enumerate(df_scaled_labeled.columns)]).T

In [36]:
df_kneighbors.head()

,month_salary,customer_monetary_value,claims_rate,premium_motor,premium_house,premium_health,premium_life,premium_work,years_as_customer,Basic,High School,BSc/MSc,PhD,No Children,Children,A1,A2,A3,A4
c_id,,,,,,,,,,,,,,,,,,,
1,-324,176,-220,431,-514,-221,104,-509,11,0,1000,0,0,0,1000,1000,0,0,0
2,-1844,-304,268,-1218,806,-505,3202,1355,19,0,1000,0,0,0,1000,0,0,0,1000
3,-223,293,-294,-506,54,-430,931,1206,0,1000,0,0,0,1000,0,0,0,1000,0
4,-1416,-197,180,-637,-655,1359,-140,-270,2,0,0,1000,0,0,1000,0,0,0,1000
5,-744,-148,120,225,-638,126,-488,3,10,0,0,1000,0,0,1000,0,0,0,1000


#### Now we have to prepare our test data as well
* Build a table with the mean and standard deviation of the train data
* Build a mask with the nulls from every column
* Create a new level for categorical variables named 'NaN'
* Replace the nulls from numeric variables with some value that would be considered impossible
* normalize all variables
* Impute one column at a time, beginning with the ones with less entropy up to more entropy

### Build table with mean and standard deviation from all variables

In [37]:
mean_df = pd.DataFrame(df_numeric.mean(), columns = ['Mean'])
standard_deviation_df = pd.DataFrame(df_numeric.std(), columns = ['Standard Deviation'])

values_df = pd.concat([mean_df, standard_deviation_df], axis=1)
values_df = values_df.T

In [38]:
new_df['Basic'].sum(), new_df['High School'].sum(), new_df['BSc/MSc'].sum(), new_df['PhD'].sum()

(1247, 3435, 4645, 669)

In [39]:
mode_degree = '3 - BSc/MSc'
mode_children = df_labels['has_children'].mode()
mode_living_area = df_labels['living_area'].mode()
print('Degree Mode: %s' % mode_degree, 
      '\nChild Mode: %.1f'%  mode_children,  
      '\nLiving Area Mode: %.1f' % mode_living_area)

Degree Mode: 3 - BSc/MSc 
Child Mode: 1.0 
Living Area Mode: 4.0


In [40]:
test_data.head()

,ed_degree,month_salary,living_area,has_children,customer_monetary_value,claims_rate,premium_motor,premium_house,premium_health,premium_life,premium_work,years_as_customer
c_id,,,,,,,,,,,,
22,2 - High School,NaN,3.0,1.0,111.37,0.80,407.52,111.70,100.13,24.67,30.34,39.0
69,2 - High School,1399.0,4.0,1.0,476.65,0.33,330.73,186.70,211.15,NaN,15.78,33.0
139,2 - High School,2538.0,2.0,1.0,149.59,0.69,194.26,305.60,NaN,37.34,17.67,37.0
144,3 - BSc/MSc,NaN,4.0,1.0,-42.45,1.02,146.36,353.95,220.04,121.80,42.01,20.0
172,3 - BSc/MSc,1086.0,4.0,1.0,NaN,NaN,378.07,78.90,166.81,6.89,18.45,26.0


In [41]:
values_df

,month_salary,customer_monetary_value,claims_rate,premium_motor,premium_house,premium_health,premium_life,premium_work,years_as_customer
Mean,2497.807365,193.012552,0.719819,297.793880,210.504862,169.414076,42.016919,41.278536,24.797119
Standard Deviation,987.324116,1063.687820,1.492800,180.838603,254.978612,104.267275,47.616092,47.848878,518.125517


### Mask with nulls from every column

In [43]:
mask_null = pd.DataFrame(index = test_data.index, columns = test_data.columns)
    
for x,y in enumerate(test_data.columns):
    mask_null.iloc[:,x] = pd.isnull(test_data.iloc[:,x])

In [44]:
mask_null.head()

,ed_degree,month_salary,living_area,has_children,customer_monetary_value,claims_rate,premium_motor,premium_house,premium_health,premium_life,premium_work,years_as_customer
c_id,,,,,,,,,,,,
22,False,True,False,False,False,False,False,False,False,False,False,False
69,False,False,False,False,False,False,False,False,False,True,False,False
139,False,False,False,False,False,False,False,False,True,False,False,False
144,False,True,False,False,False,False,False,False,False,False,False,False
172,False,False,False,False,True,True,False,False,False,False,False,False


#### Replace nulls with an implausible value

In [45]:
test_no_nulls = test_data.copy()
mean_df = mean_df.T

In [46]:
test_no_nulls['ed_degree'][mask_null['ed_degree'] == True] = mode_degree
test_no_nulls['month_salary'][mask_null['month_salary'] == True] = mean_df.loc['Mean','month_salary']
test_no_nulls['living_area'][mask_null['living_area'] == True] = int(mode_living_area)
test_no_nulls['has_children'][mask_null['has_children'] == True] = float(mode_children)
test_no_nulls['customer_monetary_value'][mask_null['customer_monetary_value'] == True] = mean_df.loc['Mean','customer_monetary_value']
test_no_nulls['claims_rate'][mask_null['claims_rate'] == True] = mean_df.loc['Mean','claims_rate']
test_no_nulls['premium_motor'][mask_null['premium_motor'] == True] = mean_df.loc['Mean','premium_motor']
test_no_nulls['premium_house'][mask_null['premium_house'] == True] = mean_df.loc['Mean','premium_house']
test_no_nulls['premium_health'][mask_null['premium_health'] == True] = mean_df.loc['Mean','premium_health']
test_no_nulls['premium_life'][mask_null['premium_life'] == True] = mean_df.loc['Mean','premium_life']
test_no_nulls['premium_work'][mask_null['premium_work'] == True] = mean_df.loc['Mean','premium_work']
test_no_nulls['years_as_customer'][mask_null['years_as_customer'] == True] = mean_df.loc['Mean','years_as_customer']
test_no_nulls

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr

,ed_degree,month_salary,living_area,has_children,customer_monetary_value,claims_rate,premium_motor,premium_house,premium_health,premium_life,premium_work,years_as_customer
c_id,,,,,,,,,,,,
22,2 - High School,2497.807365,3.0,1.0,111.370000,0.800000,407.52,111.70,100.130000,24.670000,30.340000,39.000000
69,2 - High School,1399.000000,4.0,1.0,476.650000,0.330000,330.73,186.70,211.150000,42.016919,15.780000,33.000000
139,2 - High School,2538.000000,2.0,1.0,149.590000,0.690000,194.26,305.60,169.414076,37.340000,17.670000,37.000000
144,3 - BSc/MSc,2497.807365,4.0,1.0,-42.450000,1.020000,146.36,353.95,220.040000,121.800000,42.010000,20.000000
172,3 - BSc/MSc,1086.000000,4.0,1.0,193.012552,0.719819,378.07,78.90,166.810000,6.890000,18.450000,26.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
10011,2 - High School,3959.000000,2.0,1.0,-128.240000,1.140000,177.81,153.35,311.280000,67.790000,35.450000,24.797119
10058,2 - High School,1569.000000,4.0,1.0,222.600000,0.550000,235.60,211.15,169.414076,32.340000,68.790000,33.000000
10143,3 - BSc/MSc,1652.000000,3.0,1.0,55.670000,0.880000,257.05,94.45,308.390000,42.016919,13.780000,30.000000


In [47]:
test_labels = test_no_nulls[['ed_degree', 'has_children', 'living_area']]
test_numeric = test_no_nulls.copy()
test_numeric = test_numeric.drop(['ed_degree', 'has_children', 'living_area'], axis=1)

In [48]:
test_labels['degree_he'] = pd.Categorical(test_labels['ed_degree'])
test_labels['area_he'] = pd.Categorical(test_labels['living_area'])
test_labels['children_he'] = pd.Categorical(test_labels['has_children'])

labels_degree = pd.get_dummies(test_labels['degree_he'], prefix = 'categ_degree')
labels_area = pd.get_dummies(test_labels['area_he'], prefix = 'categ_area')
labels_child = pd.get_dummies(test_labels['children_he'], prefix = 'categ_child')
test_labels = pd.concat([labels_degree, labels_child, labels_area], axis = 1)
test_labels.columns = ['Basic', 'High School', 'BSc/MSc', 'PhD', 'No Children', 'Children', 'A1', 'A2', 'A3', 'A4']
test_labels.head()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

,Basic,High School,BSc/MSc,PhD,No Children,Children,A1,A2,A3,A4
c_id,,,,,,,,,,
22,0,1,0,0,0,1,0,0,1,0
69,0,1,0,0,0,1,0,0,0,1
139,0,1,0,0,0,1,0,1,0,0
144,0,0,1,0,0,1,0,0,0,1
172,0,0,1,0,0,1,0,0,0,1


#### Standardize Data

In [56]:
normt = pd.DataFrame(index = test_numeric.index, columns = test_numeric.columns)
for i,j in enumerate(normt):
    normt.iloc[:,i] = test_numeric.iloc[:,i].apply(lambda x: (x-values_df.iloc[0,i])/values_df.iloc[1,i])

##### Notice that the values that used to be nan's are now zero - so the formula is being applied

In [57]:
normt.head()

,month_salary,customer_monetary_value,claims_rate,premium_motor,premium_house,premium_health,premium_life,premium_work,years_as_customer
c_id,,,,,,,,,
22,0.000000,-0.076754,0.053712,0.606763,-0.387503,-0.664485,-0.364308,-0.228606,0.027412
69,-1.112915,0.266655,-0.261133,0.182130,-0.093360,0.400278,0.000000,-0.532897,0.015832
139,0.040709,-0.040823,-0.019975,-0.572521,0.372953,0.000000,-0.098221,-0.493398,0.023552
144,0.000000,-0.221364,0.201086,-0.837398,0.562577,0.485540,1.675549,0.015287,-0.009259
172,-1.429933,0.000000,0.000000,0.443910,-0.516141,-0.024975,-0.737711,-0.477097,0.002322


In [58]:
test_model = pd.concat([normt, test_labels], axis = 1)
test_model.head()

,month_salary,customer_monetary_value,claims_rate,premium_motor,premium_house,premium_health,premium_life,premium_work,years_as_customer,Basic,High School,BSc/MSc,PhD,No Children,Children,A1,A2,A3,A4
c_id,,,,,,,,,,,,,,,,,,,
22,0.000000,-0.076754,0.053712,0.606763,-0.387503,-0.664485,-0.364308,-0.228606,0.027412,0,1,0,0,0,1,0,0,1,0
69,-1.112915,0.266655,-0.261133,0.182130,-0.093360,0.400278,0.000000,-0.532897,0.015832,0,1,0,0,0,1,0,0,0,1
139,0.040709,-0.040823,-0.019975,-0.572521,0.372953,0.000000,-0.098221,-0.493398,0.023552,0,1,0,0,0,1,0,1,0,0
144,0.000000,-0.221364,0.201086,-0.837398,0.562577,0.485540,1.675549,0.015287,-0.009259,0,0,1,0,0,1,0,0,0,1
172,-1.429933,0.000000,0.000000,0.443910,-0.516141,-0.024975,-0.737711,-0.477097,0.002322,0,0,1,0,0,1,0,0,0,1


In [59]:
test_kneighbors = pd.DataFrame([test_model.iloc[:,i].apply(lambda x: int(x*1000)) for i,j in enumerate(test_model.columns)]).T
test_kneighbors.head()

,month_salary,customer_monetary_value,claims_rate,premium_motor,premium_house,premium_health,premium_life,premium_work,years_as_customer,Basic,High School,BSc/MSc,PhD,No Children,Children,A1,A2,A3,A4
c_id,,,,,,,,,,,,,,,,,,,
22,0,-76,53,606,-387,-664,-364,-228,27,0,1000,0,0,0,1000,0,0,1000,0
69,-1112,266,-261,182,-93,400,0,-532,15,0,1000,0,0,0,1000,0,0,0,1000
139,40,-40,-19,-572,372,0,-98,-493,23,0,1000,0,0,0,1000,0,1000,0,0
144,0,-221,201,-837,562,485,1675,15,-9,0,0,1000,0,0,1000,0,0,0,1000
172,-1429,0,0,443,-516,-24,-737,-477,2,0,0,1000,0,0,1000,0,0,0,1000


In [60]:
clf = KNeighborsClassifier(5, weights='distance', metric = 'manhattan')
trained_model = clf.fit(df_kneighbors.loc[:,['month_salary', 'customer_monetary_value', 'claims_rate', 'premium_motor',
                                          'premium_house', 'premium_health', 'premium_life', 'premium_work', 'years_as_customer', 'Basic',
                                         'High School', 'BSc/MSc', 'PhD', 'A1', 'A2', 'A3', 'A4']],
                        df_kneighbors.loc[:,'Children'])

In [62]:
imputed_values = pd.DataFrame(index = test_kneighbors.index, columns = test_kneighbors.columns)

In [63]:
impute_children = trained_model.predict(test_kneighbors.drop(columns = ['Children', 'No Children']))
test_kneighbors['Children'][mask_null['has_children']==True] = impute_children[mask_null['has_children']==True]

In [64]:
test_kneighbors = test_kneighbors.drop(['No Children'], axis=1)
clf = KNeighborsClassifier(5, weights='distance', metric = 'manhattan')
trained_model = clf.fit(df_kneighbors.loc[:,['month_salary', 'customer_monetary_value', 'claims_rate', 'premium_motor',
                                          'premium_house', 'premium_health', 'premium_life', 'premium_work', 'years_as_customer', 'Basic',
                                         'High School', 'BSc/MSc', 'PhD', 'A2', 'A3', 'A4', 'Children']],
                        df_kneighbors.loc[:,'A1'])

imputed_area1 = trained_model.predict(test_kneighbors.drop(columns = 'A1'))
test_kneighbors['A1'][mask_null['living_area']==True] = imputed_area1[mask_null['living_area']==True]

In [65]:
test_kneighbors.head()

,month_salary,customer_monetary_value,claims_rate,premium_motor,premium_house,premium_health,premium_life,premium_work,years_as_customer,Basic,High School,BSc/MSc,PhD,Children,A1,A2,A3,A4
c_id,,,,,,,,,,,,,,,,,,
22,0,-76,53,606,-387,-664,-364,-228,27,0,1000,0,0,1000,0,0,1000,0
69,-1112,266,-261,182,-93,400,0,-532,15,0,1000,0,0,1000,0,0,0,1000
139,40,-40,-19,-572,372,0,-98,-493,23,0,1000,0,0,1000,0,1000,0,0
144,0,-221,201,-837,562,485,1675,15,-9,0,0,1000,0,1000,0,0,0,1000
172,-1429,0,0,443,-516,-24,-737,-477,2,0,0,1000,0,1000,0,0,0,1000


In [66]:
clf = KNeighborsClassifier(5, weights='distance', metric = 'manhattan')
trained_model = clf.fit(df_kneighbors.loc[:,['month_salary', 'customer_monetary_value', 'claims_rate', 'premium_motor',
                                          'premium_house', 'premium_health', 'premium_life', 'premium_work', 'years_as_customer', 'Basic',
                                         'High School', 'BSc/MSc', 'PhD', 'A1', 'A3', 'A4', 'Children']],
                        df_kneighbors.loc[:,'A2'])

impute_area2 = trained_model.predict(test_kneighbors.drop(columns = 'A2'))
test_kneighbors['A2'][mask_null['living_area']==True] = impute_area2[mask_null['living_area']==True]

In [67]:
clf = KNeighborsClassifier(5, weights='distance', metric = 'manhattan')
trained_model = clf.fit(df_kneighbors.loc[:,['month_salary', 'customer_monetary_value', 'claims_rate', 'premium_motor',
                                          'premium_house', 'premium_health', 'premium_life', 'premium_work', 'years_as_customer', 'Basic',
                                         'High School', 'BSc/MSc', 'PhD', 'A1', 'A2', 'A4', 'Children']],
                        df_kneighbors.loc[:,'A3'])

impute_area3 = trained_model.predict(test_kneighbors.drop(columns = 'A3'))
test_kneighbors['A3'][mask_null['living_area']==True] = impute_area3[mask_null['living_area']==True]

In [68]:
clf = KNeighborsClassifier(5, weights='distance', metric = 'manhattan')
trained_model = clf.fit(df_kneighbors.loc[:,['month_salary', 'customer_monetary_value', 'claims_rate', 'premium_motor',
                                          'premium_house', 'premium_health', 'premium_life', 'premium_work', 'years_as_customer', 'Basic',
                                         'High School', 'BSc/MSc', 'PhD', 'A1', 'A3', 'A2', 'Children']],
                        df_kneighbors.loc[:,'A4'])

impute_area4 = trained_model.predict(test_kneighbors.drop(columns = 'A4'))
test_kneighbors['A4'][mask_null['living_area']==True] = impute_area4[mask_null['living_area']==True]

In [69]:
clf = KNeighborsClassifier(5, weights='distance', metric = 'manhattan')
trained_model = clf.fit(df_kneighbors.loc[:,['month_salary', 'customer_monetary_value', 'claims_rate', 'premium_motor',
                                          'premium_house', 'premium_health', 'premium_life', 'premium_work', 'years_as_customer',
                                         'High School', 'BSc/MSc', 'PhD', 'A1', 'A3', 'A2', 'A4', 'Children']],
                        df_kneighbors.loc[:,'Basic'])

impute_education = trained_model.predict(test_kneighbors.drop(columns = 'Basic'))
test_kneighbors['Basic'][mask_null['ed_degree']==True] = impute_education[mask_null['ed_degree']==True]

In [70]:
clf = KNeighborsClassifier(5, weights='distance', metric = 'manhattan')
trained_model = clf.fit(df_kneighbors.loc[:,['month_salary', 'customer_monetary_value', 'claims_rate', 'premium_motor',
                                          'premium_house', 'premium_health', 'premium_life', 'premium_work', 'years_as_customer', 'Basic',
                                         'BSc/MSc', 'PhD', 'A1', 'A3', 'A2', 'A4', 'Children']],
                        df_kneighbors.loc[:,'High School'])

impute_education = trained_model.predict(test_kneighbors.drop(columns = 'High School'))
test_kneighbors['High School'][mask_null['ed_degree']==True] = impute_education[mask_null['ed_degree']==True]

In [71]:
clf = KNeighborsClassifier(5, weights='distance', metric = 'manhattan')
trained_model = clf.fit(df_kneighbors.loc[:,['month_salary', 'customer_monetary_value', 'claims_rate', 'premium_motor',
                                          'premium_house', 'premium_health', 'premium_life', 'premium_work', 'years_as_customer', 'Basic',
                                         'High School', 'PhD', 'A1', 'A3', 'A2', 'A4', 'Children']],
                        df_kneighbors.loc[:,'BSc/MSc'])

impute_education = trained_model.predict(test_kneighbors.drop(columns = 'BSc/MSc'))
test_kneighbors['BSc/MSc'][mask_null['ed_degree']==True] = impute_education[mask_null['ed_degree']==True]

In [72]:
clf = KNeighborsClassifier(5, weights='distance', metric = 'manhattan')
trained_model = clf.fit(df_kneighbors.loc[:,['month_salary', 'customer_monetary_value', 'claims_rate', 'premium_motor',
                                          'premium_house', 'premium_health', 'premium_life', 'premium_work', 'years_as_customer', 'Basic',
                                         'High School', 'BSc/MSc', 'A1', 'A3', 'A2', 'A4', 'Children']],
                        df_kneighbors.loc[:,'PhD'])

impute_education = trained_model.predict(test_kneighbors.drop(columns = 'PhD'))
test_kneighbors['PhD'][mask_null['ed_degree']==True] = impute_education[mask_null['ed_degree']==True]

In [73]:
values_df.loc['Mean',:]/values_df.loc['Standard Deviation',:]

month_salary               2.529876
customer_monetary_value    0.181456
claims_rate                0.482194
premium_motor              1.646738
premium_house              0.825579
premium_health             1.624806
premium_life               0.882410
premium_work               0.862686
years_as_customer          0.047859
dtype: float64

In [74]:
clf = KNeighborsClassifier(5, weights='distance', metric = 'manhattan')
trained_model = clf.fit(df_kneighbors.loc[:,['month_salary', 'customer_monetary_value', 'claims_rate', 'premium_motor',
                                          'premium_house', 'premium_health', 'premium_life', 'years_as_customer', 'Basic',
                                         'High School', 'BSc/MSc', 'PhD', 'A1', 'A3', 'A2', 'A4', 'Children']],
                        df_kneighbors.loc[:,'premium_work'])

impute_work = trained_model.predict(test_kneighbors.drop(columns = 'premium_work'))
test_kneighbors['premium_work'][mask_null['premium_work']==True] = impute_work[mask_null['premium_work']==True]

In [75]:
clf = KNeighborsClassifier(5, weights='distance', metric = 'manhattan')
trained_model = clf.fit(df_kneighbors.loc[:,['month_salary', 'customer_monetary_value', 'claims_rate', 'premium_motor',
                                          'premium_health', 'premium_life', 'premium_work', 'years_as_customer', 'Basic',
                                         'High School', 'BSc/MSc', 'PhD', 'A1', 'A3', 'A2', 'A4', 'Children']],
                        df_kneighbors.loc[:,'premium_house'])

impute_house = trained_model.predict(test_kneighbors.drop(columns = 'premium_house'))
test_kneighbors['premium_house'][mask_null['premium_house']==True] = impute_house[mask_null['premium_house']==True]

In [76]:
clf = KNeighborsClassifier(5, weights='distance', metric = 'manhattan')
trained_model = clf.fit(df_kneighbors.loc[:,['month_salary', 'customer_monetary_value', 'claims_rate', 'premium_motor',
                                          'premium_health', 'premium_house', 'premium_work', 'years_as_customer', 'Basic',
                                         'High School', 'BSc/MSc', 'PhD', 'A1', 'A3', 'A2', 'A4', 'Children']],
                        df_kneighbors.loc[:,'premium_life'])

impute_life = trained_model.predict(test_kneighbors.drop(columns = 'premium_life'))
test_kneighbors['premium_life'][mask_null['premium_life']==True] = impute_life[mask_null['premium_life']==True]

In [77]:
clf = KNeighborsClassifier(5, weights='distance', metric = 'manhattan')
trained_model = clf.fit(df_kneighbors.loc[:,['month_salary', 'customer_monetary_value', 'premium_life', 'premium_motor',
                                          'premium_health', 'premium_house', 'premium_work', 'years_as_customer', 'Basic',
                                         'High School', 'BSc/MSc', 'PhD', 'A1', 'A3', 'A2', 'A4', 'Children']],
                        df_kneighbors.loc[:,'claims_rate'])

impute_claims = trained_model.predict(test_kneighbors.drop(columns = 'claims_rate'))
test_kneighbors['claims_rate'][mask_null['claims_rate']==True] = impute_claims[mask_null['claims_rate']==True]

In [78]:
clf = KNeighborsClassifier(5, weights='distance', metric = 'manhattan')
trained_model = clf.fit(df_kneighbors.loc[:,['month_salary', 'customer_monetary_value', 'premium_life', 'claims_rate',
                                          'premium_health', 'premium_house', 'premium_work', 'years_as_customer', 'Basic',
                                         'High School', 'BSc/MSc', 'PhD', 'A1', 'A3', 'A2', 'A4', 'Children']],
                        df_kneighbors.loc[:,'premium_motor'])

impute_motor = trained_model.predict(test_kneighbors.drop(columns = 'premium_motor'))
test_kneighbors['premium_motor'][mask_null['premium_motor']==True] = impute_motor[mask_null['premium_motor']==True]

In [79]:
clf = KNeighborsClassifier(5, weights='distance', metric = 'manhattan')
trained_model = clf.fit(df_kneighbors.loc[:,['month_salary', 'customer_monetary_value', 'premium_life', 'claims_rate',
                                          'premium_motor', 'premium_house', 'premium_work', 'years_as_customer', 'Basic',
                                         'High School', 'BSc/MSc', 'PhD', 'A1', 'A3', 'A2', 'A4', 'Children']],
                        df_kneighbors.loc[:,'premium_health'])

impute_health = trained_model.predict(test_kneighbors.drop(columns = 'premium_health'))
test_kneighbors['premium_health'][mask_null['premium_health']==True] = impute_health[mask_null['premium_health']==True]

In [80]:
clf = KNeighborsClassifier(5, weights='distance', metric = 'manhattan')
trained_model = clf.fit(df_kneighbors.loc[:,['premium_health', 'customer_monetary_value', 'premium_life', 'claims_rate',
                                          'premium_motor', 'premium_house', 'premium_work', 'years_as_customer', 'Basic',
                                         'High School', 'BSc/MSc', 'PhD', 'A1', 'A3', 'A2', 'A4', 'Children']],
                        df_kneighbors.loc[:,'month_salary'])

impute_salary = trained_model.predict(test_kneighbors.drop(columns = 'month_salary'))
test_kneighbors['month_salary'][mask_null['month_salary']==True] = impute_salary[mask_null['month_salary']==True]

In [81]:
clf = KNeighborsClassifier(5, weights='distance', metric = 'manhattan')
trained_model = clf.fit(df_kneighbors.loc[:,['premium_health', 'customer_monetary_value', 'premium_life', 'claims_rate',
                                          'premium_motor', 'premium_house', 'premium_work', 'month_salary', 'Basic',
                                         'High School', 'BSc/MSc', 'PhD', 'A1', 'A3', 'A2', 'A4', 'Children']],
                        df_kneighbors.loc[:,'years_as_customer'])

impute_years_customer = trained_model.predict(test_kneighbors.drop(columns = 'years_as_customer'))
test_kneighbors['years_as_customer'][mask_null['years_as_customer']==True] = impute_years_customer[mask_null['years_as_customer']==True]

In [82]:
complete_data = pd.concat([df_kneighbors, test_kneighbors], axis=0)

In [83]:
#complete_data = complete_data.drop(['No Children'], axis=1)
complete_data2 = pd.DataFrame([complete_data.iloc[:,i].apply(lambda x: float(x/1000)) for i,j in enumerate(complete_data.columns)]).T

In [84]:
complete_data2.to_pickle('complete_data.pkl')

In [85]:
all_data = complete_data2.copy()

all_data['education'] = 16
all_data['education'][all_data['Basic']==1.0] = 4
all_data['education'][all_data['High School']==1.0] = 12
all_data['education'][all_data['PhD']==1.0] = 21

all_data['living_area']=1
all_data['living_area'][all_data['A2']==1.0] = 2
all_data['living_area'][all_data['A3']==1.0] = 3
all_data['living_area'][all_data['A4']==1.0] = 4

all_data.head()

/usr/lib/python3/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/lib/python3/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/lib/python3/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value i

,month_salary,customer_monetary_value,claims_rate,premium_motor,premium_house,premium_health,premium_life,premium_work,years_as_customer,Basic,...,BSc/MSc,PhD,No Children,Children,A1,A2,A3,A4,education,living_area
c_id,,,,,,,,,,,,,,,,,,,,,
1,-0.324,0.176,-0.220,0.431,-0.514,-0.221,0.104,-0.509,0.011,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,12,1
2,-1.844,-0.304,0.268,-1.218,0.806,-0.505,3.202,1.355,0.019,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,12,4
3,-0.223,0.293,-0.294,-0.506,0.054,-0.430,0.931,1.206,0.000,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,4,3
4,-1.416,-0.197,0.180,-0.637,-0.655,1.359,-0.140,-0.270,0.002,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,16,4
5,-0.744,-0.148,0.120,0.225,-0.638,0.126,-0.488,0.003,0.010,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,16,4


In [86]:
all_data = all_data.drop(['A1','A2','A3','A4','BSc/MSc', 'Basic', 'High School', 'PhD', 'No Children'], axis=1)

In [87]:
saved_data = all_data.copy()

In [88]:
all_data = saved_data.copy()
k = []
for i,j in enumerate(all_data.columns):
    k.append(j)

a = ['Children', 'living_area', 'education']

for i in a:
    k.remove(i)    

for i in k:
    all_data[i] *= values_df.loc['Standard Deviation',i]
    all_data[i] += values_df.loc['Mean',i]

In [89]:
all_data.head()

,month_salary,customer_monetary_value,claims_rate,premium_motor,premium_house,premium_health,premium_life,premium_work,years_as_customer,Children,education,living_area
c_id,,,,,,,,,,,,
1,2177.914351,380.221608,0.391403,375.735317,79.445855,146.371008,46.968992,16.923458,30.496500,1.0,12,1
2,677.181694,-130.348545,1.119889,77.532461,416.017623,116.759102,194.483645,106.113766,34.641504,1.0,12,4
3,2277.634087,504.673083,0.280936,206.289547,224.273707,124.579148,86.347500,98.984283,24.797119,0.0,4,3
4,1099.756416,-16.533948,0.988523,182.599690,43.493871,311.113302,35.350666,28.359339,25.833370,1.0,16,4
5,1763.238222,35.586755,0.898955,338.482565,47.828508,182.551752,18.780266,41.422083,29.978374,1.0,16,4


In [90]:
all_data.to_pickle('all_data.pkl')